# Plotting notebook for moremelt climatology runs
## Set up
### Packages

In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
import pandas as pd
import scipy
from scipy import stats
import matplotlib as mpl
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.mathtext import _mathtext as mathtext
import matplotlib.ticker as mticker
from matplotlib import font_manager
from matplotlib import gridspec, animation
import matplotlib.path as mpath
import matplotlib.colors as colors
import matplotlib.dates as mdates
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import warnings
warnings.simplefilter('ignore', UserWarning)
warnings.filterwarnings('ignore')
import datetime as dt
from datetime import timedelta
from cmcrameri import cm
import jinja2
from Plotting_functions import wvl2wvn, wvn2wvl, p2z, z2p, t_test_two_means, Wilks_pcrit, CustomCmap, draw_circle, CalcStatSig
import cftime
import dask
from dask_jobqueue import PBSCluster
from dask.distributed import Client
from functools import partial
from collections import defaultdict

In [2]:
font_path = '/glade/work/glydia/conda-envs/cenv/fonts/Helvetica.ttc'  # Your font path goes here
font_manager.fontManager.addfont(font_path)
prop = font_manager.FontProperties(fname=font_path)

mpl.rcParams['font.family'] = 'sans-serif'
mpl.rcParams['font.sans-serif'] = 'Helvetica'

### Filepaths and name variables

In [3]:
## Plot types to make - CHANGE
# 0: Weighted spatial mean or sea ice area
# 1: Volume
# 2: Leave alone (doing spatial map or sea ice concentration)
plots = {
    'map': [True, 2],
    'ts': [False, 0],
    'siev': [False, 0],
}

## Categorical plot type - DO NOT CHANGE
plot_types = {
    'spatial': plots['map'][0],
    'line': plots['ts'][0] or plots['siev'][0]
}

# Spatial & time domain - CHANGE s_domain & t_domain only
s_domain = 1 # 0: Global, 1: Arctic, 2: Antarctic
a_domain = plot_types['spatial'] # True: 50-90, False: 70-90
t_domain = 911 # start year

## Time averaging type - CHANGE
time_avg = 0   # 0: Monthly, 1: Yearly, 2: Seasonal, 3: All data, 4: Timeseries

# Variables - CHANGE
comp = 'ice'
var_ind = 1

# DO NOT CHANGE
var_list = {'atm': ['TREFHT'],
            'ice': ['aice', 'hi', 'hs']}
var = var_list[comp][var_ind]

# CHANGE
type_onemonth = 9 # Selects single month to plot timeseries for (time_avg = 4), if doing all months, equals 0
plot_levels = [300,500, 850,925] # Selects plot levels for vlev type plots

In [4]:
## Test numbers - DO NOT CHANGE
tst_nums = np.arange(1,4)

## Test names - True means plot, False means don't plot
# CHANGE
ds_plot_list = {
    'LENS2 piControl': True,
    'lessmelt piControl': False,
    'moremelt_rsnw0': True
}

## Filepaths - DO NOT CHANGE
path_to_data = '/glade/work/glydia/Arctic_controls_processed_data/climo_plotting_data/'
path_to_graphs = '/glade/u/home/glydia/moremelt_climo_graphs/'

# Conditions - DO NOT CHANGE
vert_lev = {'atm': [False],
            'ice': [False,False,False]}
file_bool = not vert_lev[comp][var_ind]

In [5]:
########################## DO NOT CHANGE ANYTHING BELOW THIS LINE #############################

In [6]:
%%time
    
## Select plot type
time_str_list = {0: 'month', 1: 'year'}
time_outstr = time_str_list[time_avg]

## Select time and spatial domain strings
sd_str_list = {0: 'Global', 1: 'Arctic', 2: 'Antarctic'}
sd_str = sd_str_list[s_domain]
td_str = str(t_domain).zfill(4)

mon_str = np.array(['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
month_str = np.array(['January','February','March','April','May','June','July','August','September','October','November','December'])

CPU times: user 31 µs, sys: 0 ns, total: 31 µs
Wall time: 33.6 µs


In [7]:
## Set up properties of each dataset and whether it will be plotted or not
# O: Single run
# attribute structure: [use dataset[0], dataset typ[1], line color[2], line style[3], zorder[4], note[5](optional)]
ds_names = {
    'LENS2 piControl': [True, 0, 'k', '-', 1, 'pic'],
    'lessmelt piControl':  [None, 0, 'blue', '-', 3, 'lmpi'],
    'moremelt_rsnw0': [None, 0, 'red', '-', 3, 'mmpi'],
}

ds_paper_names = {
    'LENS2 piControl': 'PI-control',
    'lessmelt piControl': 'PI-lessmelt',
    'moremelt_rsnw0': 'PI-moremelt',
}

# Update PiC_UVnudge 'use dataset' values with T/F from ds_plot_list
for dsname, use in ds_plot_list.items():
    ds_names[dsname][0] = use

In [8]:
## Determine note type based on model runs plotted
note = ''

for dsname, attrs in ds_names.items():
    # Only pick datasets that will plotted and that have a code
    if attrs[0] and len(attrs) == 6:
        # Each PiC_UVnudge run has its own code, if the run will be plotted, it's code will be added to the note
        note = note+attrs[5]+'-'

note = note[:-1]

### Custom functions

In [9]:
def LoadData(plot_type, varname, tavg, plot_level=None):
    # Create file name
    filepath = path_to_data+plot_type+'.'+varname+'.'+sd_str+'.'+td_str+'.'+tavg+'.nc'
    
    data = xr.open_dataset(filepath)
    
    return data

In [10]:
def SubCmap(orgcmap, levels, type, color):
    # Type is middle, end or beginning
    # Make cmap values swapped out
    # color is color to sub in
    lev_len = len(levels)+1
    cmap_samp = orgcmap.resampled(lev_len)
    cmap_list = cmap_samp(np.linspace(0,1,lev_len))

    # Swap out first two
    if type == 'beg':
        lev_ind = 0
        cmap_list[lev_ind:(lev_ind+1), :] = color[0]
        cmap_list[(lev_ind+1):(lev_ind+2), :] = color[1]
        final_cmap, _ = CustomCmap(levels, cmap_list, 
                           [color[0], orgcmap.get_over()], True)

    # Swap out middle two
    elif type == 'mid':
        lev_ind = int(lev_len/2)
        cmap_list[(lev_ind-1):(lev_ind+1), :] = color
        final_cmap, _ = CustomCmap(levels, cmap_list, 
                           [orgcmap.get_under(), orgcmap.get_over()], True)

    # Swap out last
    else:
        lev_ind = -1
        cmap_list[lev_ind:, :] = color
        final_cmap, _ = CustomCmap(levels, cmap_list, 
                           [orgcmap.get_under(), color], True)

    return final_cmap

In [11]:
def AxisLabels(ax, title_str, land):
    if land:
        ax.add_feature(cfeature.LAND, zorder=3)
    ax.coastlines(zorder=4)
    ax.set_extent(extent, ccrs.PlateCarree())
    ax.set_title(title_str, fontsize = 12)
    draw_circle(ax, draw_circ=(s_domain >= 1), draw_major=False)
    return None

In [12]:
def SaveFig(fig, plot_type, tavg, varname, note=None, plot_level=None):
    # File format is:
    # var.ensemble_type.spatialdomain.[Zplot_level].plot_type.timeaveraging.timedomain.[note].png
    level_str = '' if plot_level == None else 'Z'+str(plot_level)+'.'
    note_str = '' if note == None else note+'.'
    
    fig.savefig(path_to_graphs+varname+'.'+sd_str+'.'+level_str+plot_type+'.'+tavg+'.'+td_str+'.'+note_str+'pdf', bbox_inches='tight')
    return None

## Line plots
### Set up

In [13]:
if plot_types['line']:
    graph_type_str = 'Linear'

    ylabelabs = {'aice': ' sea ice extent (million km$^2$)',
                 'hi': ' sea ice volume (10$^{13}$ m$^3$)',
                 'TREFHT': ' 2m air temperature (K)'}

    wdth = {911: 6}

    legend_loc = {0: (0.6, 0.7),
                  1: (0.3, 0.75)}
    
    
    # Prepare x ticks
    # Monthly cycle
    if time_avg == 0:
        xtick_loc = np.arange(0,12,1)
        xtick_loc_min = np.arange(0,12,1)
        xtick_lbl = mon_str
        xlim = [0,11]
        date_str = 'month'
        period ='month'
        xlabel = ''

    # Yearly
    elif time_avg == 1:
        xtick_loc = np.arange(t_domain,t_domain+200,25)
        xtick_loc_min = np.arange(t_domain,t_domain+200,5)
        xtick_lbl = np.arange(t_domain,t_domain+200,25)
        xlim = [t_domain,t_domain+199]
        date_str = 'timeseries_yr'
        period='year'
        xlabel= 'Year'
        dim_avg = 'time.year'

### Data loading

In [14]:
%%time

if plot_types['line']:

    ## Absolute 
    if comp == 'atm':
        ds_abs = LoadData(graph_type_str+'.abs', var, period)
        sbpt_shp = (1,1)
        figsz = (6,3.5)

    elif comp == 'ice':
        ds_abse = LoadData(graph_type_str+'.abs', 'aice', period)
        ds_absv = LoadData(graph_type_str+'.abs', 'hi', period)
        ds_abs = [ds_abse, ds_absv]
        ice_vars = ['aice','hi']
        sbpt_shp = (2,1)
        let_list = ['a)','b)']
        if time_avg == 1:
            figsz = (6,7)
        elif time_avg == 0:
            figsz = (5,4.5)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


### Make absolute plots

In [15]:
%%time

if plot_types['line'] and comp == 'atm':
    # Setup figure
    fig,ax = plt.subplots(sbpt_shp[0],sbpt_shp[1], layout='constrained')
    fig.set_size_inches(figsz[0],figsz[1])

    for dsname, attrs in ds_names.items():
        if attrs[0]:
            da_abs = ds_abs[dsname]

            # Plot absolute value
            da_abs.plot(
                ax=ax, color=attrs[2], ls=attrs[3], x=period, zorder=attrs[4], label=ds_paper_names[dsname])
            
    # Formatting
    ax.set_xlabel(xlabel)
    ax.set_xlim(xlim)
    ax.set_xticks(ticks=xtick_loc,labels=xtick_lbl)
    ax.set_xticks(ticks=xtick_loc_min,minor=True)
    ax.set_ylabel(sd_str+ylabelabs[var])
    ax.legend(fontsize=8, edgecolor='w')
    ax.set_title('')
    ax.grid(alpha=0.3)

    SaveFig(fig, graph_type_str+'.abs', date_str, var, note)

    plt.close(fig)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 5.96 µs


In [16]:
%%time

if plot_types['line'] and comp == 'ice':
    # Setup figure
    fig, axlist = plt.subplots(sbpt_shp[0],sbpt_shp[1], layout='constrained')
    fig.set_size_inches(figsz[0],figsz[0])

    for i in range(0,len(ds_abs)):
        ds = ds_abs[i]
        ax = axlist[i]
        vari = ice_vars[i]
        
        for dsname, attrs in ds_names.items():
            if attrs[0]:
                da_abs = ds[dsname]
    
                # Plot absolute value, sie then siv
                da_abs.plot(
                    ax=ax, color=attrs[2], ls=attrs[3], x=period, zorder=attrs[4], label=ds_paper_names[dsname])
    
        # Formatting
        ax.set_xlabel(xlabel)
        ax.set_xlim(xlim)
        ax.set_xticks(ticks=xtick_loc,labels=xtick_lbl)
        ax.set_xticks(ticks=xtick_loc_min,minor=True)
        ax.set_ylabel(sd_str+ylabelabs[vari])
        ax.legend(fontsize=8, edgecolor='w',loc=legend_loc[time_avg])
        ax.set_title('')
        ax.grid(alpha=0.3)
        ax.annotate(let_list[i],(0.1,0.1),xytext=(-10,0),ha='right',va='center',
                    size=12,xycoords='axes fraction',textcoords='offset points')

    SaveFig(fig, graph_type_str+'.abs', date_str, 'siev', note)

    plt.close(fig)
    

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 5.48 µs


## Spatial plots
### Set up

In [17]:
if plot_types['spatial']:
    if s_domain == 1:
        proj = ccrs.NorthPolarStereo()
        extent = [-180, 180, 50, 90]
    elif s_domain == 2:
        proj = ccrs.SouthPolarStereo()
        extent = [-180, 180, -90, -50]

    # Monthly
    if time_avg == 0:
        date_str = mon_str
        period = 'time'
    
    # Latitude, longitude, and level names (ERA5 and the CESM produced datasets have different names)
    xname = 'lon'
    yname = 'lat'

    # Determine width of plots
    wdth = 2
    hgt = 2 
    
    # Plot letter
    plot_let_full = np.array(['a) ', 'b) ', 'c) ', 'd) '])
    plot_let = plot_let_full[0:(wdth*hgt)].reshape((hgt, wdth))

    # Do we add land features
    addland = (comp == 'ice')

In [18]:
if plots['map'][0]:
    graph_type_str = 'Map'

    # Arctic min: Sept, max: March
    if s_domain == 1:
        index_list = np.array(['Sep','Mar'])
        index_name = 'month'
    # Antarctic min: Feb, max: Sept
    elif s_domain == 2:
        index_list = np.array(['Feb','Sep'])
        index_name = 'month'
        
    figsz = (4*wdth,4*hgt)
    sbpt_shp = (hgt,wdth)

    # Plotting variables
    mlevels = {'hi': np.array([0.0,0.01,0.10,0.50,1.0,1.5,2.0,2.5,3.0,4.0,5.0]),
               'aice': np.array([0.0,0.05,0.10,0.15,0.20,0.30,0.40,0.50,0.60,0.70,0.80,0.90,0.95,1.00])}
    mdlevels = {'hi': np.array([-1.5,-1.25,-1.0,-0.75,-0.5,-0.25,-0.01,0.0,0.01,0.25,0.5,0.75,1.0,1.25,1.5]),
               'aice': np.array([-0.4,-0.3,-0.2,-0.15,-0.1,-0.01,0.0,0.01,0.1,0.15,0.2,0.3,0.4])}
    mcmaps = {'hi': cm.devon,
              'aice': cm.devon}
    mdcmaps = {'hi': SubCmap(cm.vik, mdlevels['hi'], 'mid', np.array([0,0,0,0])),
               'aice': SubCmap(cm.vik, mdlevels['aice'], 'mid', np.array([0,0,0,0]))}
    mlabels = {'hi': ' sea ice thickness (m)',
               'aice': ' sea ice concetration'}   

### Data loading

In [19]:
%%time

if plots['map'][0]:
    # Load spatial averages
    ds_sp = LoadData(graph_type_str, var, time_outstr)

CPU times: user 982 ms, sys: 226 ms, total: 1.21 s
Wall time: 1.28 s


### Make plots

In [20]:
%%time

if plots['map'][0] and file_bool:
    
    # Plotting of spatial patterns for sea ice concentration & thickness
    # Setup figure
    fig, axlist = plt.subplots(sbpt_shp[0], sbpt_shp[1], layout='constrained', subplot_kw=dict(projection=proj))
    fig.set_size_inches(figsz[0],figsz[1])
    
    for i in np.arange(0,len(index_list)):
        # Time index
        t_index = {index_name:index_list[i]}

        dm_sp = ds_sp['moremelt_rsnw0'].loc[t_index]
        dc_sp = ds_sp['LENS2 piControl'].loc[t_index]
        dd_sp = dm_sp-dc_sp

        # Plot moremelt data
        cax = dm_sp.plot.contourf(
                ax=axlist[i,0], x=xname, y=yname, cmap=mcmaps[var], 
                levels=mlevels[var], add_colorbar=False,transform=ccrs.PlateCarree(),zorder=1)
        
        # Plot difference data
        caxd = dd_sp.plot.contourf(
                ax=axlist[i,1], x=xname, y=yname, cmap=mdcmaps[var], 
                levels=mdlevels[var], add_colorbar=False,transform=ccrs.PlateCarree(),zorder=1)
        
        # Formatting
        AxisLabels(axlist[i,0], plot_let[i,0]+ds_paper_names['moremelt_rsnw0'], addland)
        AxisLabels(axlist[i,1], plot_let[i,1]+ds_paper_names['moremelt_rsnw0']+' — '+ds_paper_names['LENS2 piControl'], addland)

        # Add colorbar
        cb = fig.colorbar(cax, ax=axlist[i,0], pad=0.1, shrink=0.75,fraction=0.1, extend='both', orientation='vertical')
        cb.set_label(label=index_list[i]+mlabels[var], fontsize=12)
        cb.ax.tick_params(labelsize=10)
        cb.set_ticks(ticks=mlevels[var])
    
        # Add difference colorbar
        cbd = fig.colorbar(caxd, ax=axlist[i,1], pad=0.1, shrink=0.75,fraction=0.1, extend='both', orientation='vertical')
        cbd.set_label(label=index_list[i]+mlabels[var], fontsize=12)
        cbd.ax.tick_params(labelsize=10)
        cbd.set_ticks(ticks=mdlevels[var])

    SaveFig(fig, graph_type_str, index_name, var, note)

    plt.close(fig)

CPU times: user 4.26 s, sys: 34.1 ms, total: 4.3 s
Wall time: 4.56 s
